# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [2]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [3]:
text = text.lower()

Разобьём весь текст на предложения. 

In [4]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

13565

In [5]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [6]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [7]:
sents_tokenize = [tokenizer.tokenize(sent) for sent in sents]
sents_tokenize[4:6]

[['и',
  'каждый',
  'раз',
  'молодой',
  'человек',
  'проходя',
  'мимо',
  'чувствовал',
  'какое',
  'то',
  'болезненное',
  'и',
  'трусливое',
  'ощущение',
  'которого',
  'стыдился',
  'и',
  'от',
  'которого',
  'морщился'],
 ['он',
  'был',
  'должен',
  'кругом',
  'хозяйке',
  'и',
  'боялся',
  'с',
  'нею',
  'встретиться']]

In [8]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [9]:
len(words) # всего слов

173403

In [10]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [11]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [12]:
sents_tokenize = [[word for word in text_cur if word not in stopwords_ru] for text_cur in sents_tokenize]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Каждый должен знать основы нейронных сетей!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'каждый должный знать основа нейронный сеть'

In [15]:
p = morph.parse('плачу')
p

[Parse(word='плачу', tag=OpencorporaTag('NOUN,inan,masc sing,datv'), normal_form='плач', score=0.3333333333333333, methods_stack=((DictionaryAnalyzer(), 'плачу', 80, 2),)),
 Parse(word='плачу', tag=OpencorporaTag('VERB,impf,intr sing,1per,pres,indc'), normal_form='плакать', score=0.3333333333333333, methods_stack=((DictionaryAnalyzer(), 'плачу', 2468, 1),)),
 Parse(word='плачу', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='платить', score=0.3333333333333333, methods_stack=((DictionaryAnalyzer(), 'плачу', 2472, 1),))]

Обработаем все слова из датасета. 

In [16]:
sents_tokenize = [[morph.normal_forms(word)[0] for word in text_cur] for text_cur in sents_tokenize]

In [17]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [18]:
len(words) # всего слов

93069

In [19]:
len(set(words)) # уникальных слов

11084

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [читай мой мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [20]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1449),
 ('всё', 963),
 ('знать', 630),
 ('раскольник', 567),
 ('свой', 549),
 ('один', 548),
 ('сказать', 544),
 ('говорить', 536),
 ('человек', 501),
 ('весь', 442),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('сам', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('петрович', 369)]

In [21]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4768

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `vector_size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [22]:
%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(vector_size=100, window=2, min_count=3, workers=4)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

CPU times: user 11.4 s, sys: 145 ms, total: 11.5 s
Wall time: 7.75 s


(7944386, 9306900)

In [26]:
model.corpus_count # число примеров в обучающей выборке

13565

## 3. Свойства модели

In [23]:
# вектор слова
model.wv['старуха'][:10]

array([ 0.02088769,  0.408562  , -0.394518  , -1.1819779 ,  0.28159457,
       -0.8527992 , -2.5708127 , -0.76846147, -1.0608165 ,  0.37799102],
      dtype=float32)

In [24]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [25]:
# похожести слов 
model.wv.similarity('тварь', 'право')

0.00345319

In [26]:
model.wv.similarity('старуха', 'топор')

0.40388972

In [27]:
model.wv.similarity('тварь', 'тварь')

1.0

In [28]:
# самые похожие
model.wv.most_similar('старуха')

[('лизавета', 0.4844273328781128),
 ('близко', 0.42552414536476135),
 ('наклепать', 0.42292913794517517),
 ('чиновница', 0.41330593824386597),
 ('топор', 0.4038897156715393),
 ('обратно', 0.39545053243637085),
 ('ухо', 0.37545090913772583),
 ('обыск', 0.37424159049987793),
 ('обух', 0.37400510907173157),
 ('выставлять', 0.3712168037891388)]

In [29]:
model.wv.most_similar('лестница')

[('ступенька', 0.5212623476982117),
 ('прихожая', 0.516538679599762),
 ('узенький', 0.5063650608062744),
 ('всходить', 0.498861700296402),
 ('цыпочки', 0.45446863770484924),
 ('ступень', 0.4509531557559967),
 ('вниз', 0.43974754214286804),
 ('ниже', 0.4364323914051056),
 ('передний', 0.43393903970718384),
 ('спускаться', 0.42598918080329895)]

In [30]:
# арифметика
model.wv.most_similar(positive=['раскольников'], negative=['тварь'])[:10]

[('несколько', 0.3130395710468292),
 ('задушить', 0.3030407726764679),
 ('прислушиваться', 0.2827918231487274),
 ('затем', 0.277862548828125),
 ('слушать', 0.27226680517196655),
 ('всматриваться', 0.26809871196746826),
 ('уставиться', 0.266798734664917),
 ('произнести', 0.2650647461414337),
 ('казённый', 0.26373594999313354),
 ('папироска', 0.2611480951309204)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [31]:
model_path = "./our_w2v.model"
model.save(model_path)

In [32]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [33]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [34]:
text2 = text2.lower()
sents2 = sent_tokenize(text2)

sents_tokenize2 = [tokenizer.tokenize(sent) for sent in sents2]
sents_tokenize2 = [[morph.normal_forms(word)[0] for word in text_cur if word not in stopwords_ru]
                      for text_cur in sents_tokenize2]


In [36]:
sents_tokenize2[10]

['сени', 'выйти', 'царь', 'отец']

In [37]:
len(sents_tokenize2)

254

Дополняем модель.

In [42]:
# обновили словарь
our_model.build_vocab(sents_tokenize2, update=True)

# дообучили
our_model.train(sents_tokenize2, total_examples=our_model.corpus_count, epochs=100)

(177393, 269000)

In [43]:
our_model.wv.most_similar('ядро')

[('изумруд', 0.6903663873672485),
 ('чистый', 0.6048203110694885),
 ('грызть', 0.5976995825767517),
 ('скатерть', 0.5967453718185425),
 ('серебро', 0.5682592391967773),
 ('шкатулка', 0.5658619403839111),
 ('орешек', 0.5432782769203186),
 ('золотой', 0.5385962128639221),
 ('стальной', 0.5213777422904968),
 ('подсвечник', 0.5162104368209839)]

Пример со старым словом.

In [44]:
our_model.wv.most_similar('сын')

[('губерния', 0.5642402172088623),
 ('обеспечить', 0.5593199133872986),
 ('благословить', 0.4838382303714752),
 ('адвокатский', 0.48227956891059875),
 ('враньё', 0.47301098704338074),
 ('поклоняться', 0.4573090076446533),
 ('воровать', 0.45300784707069397),
 ('сумление', 0.4487994313240051),
 ('особо', 0.4466111958026886),
 ('уезд', 0.4458710253238678)]

In [45]:
model.wv.most_similar('сын')

[('обеспечить', 0.4880862832069397),
 ('губерния', 0.4670483469963074),
 ('адвокатский', 0.4602300524711609),
 ('благословить', 0.45936816930770874),
 ('детство', 0.45760074257850647),
 ('приём', 0.45006826519966125),
 ('расспрос', 0.4476972818374634),
 ('враньё', 0.4467470347881317),
 ('преувеличить', 0.44639700651168823),
 ('следить', 0.4461674392223358)]